In [12]:
# ロジスティック回帰のROC曲線

require 'daru'
require 'nyaplot'
require 'numo/narray'

Variances = [50, 150] # 両クラス共通の分散（2種類の分散で計算を実施）

def normal_rand(mu = 0, sigma = 1.0)
  random = Random.new
  (Math.sqrt(-2 * Math.log(random.rand)) * Math.sin(2 * Math::PI * random.rand) * sigma) + mu
end

# データセット {x_n,y_n,type_n} を用意
def prepare_dataset(variance)
  n1 = 80
  n2 = 200
  mu1 = [9, 9]
  mu2 = [-3, -3]

  sigma = Math.sqrt(variance)

  df1 = n1.times.map do
    [normal_rand(mu1[0], sigma), normal_rand(mu1[1], sigma)]
  end
  df1 = df1.transpose
  df1 = Daru::DataFrame.new(x: df1[0], y: df1[1], type: Array.new(n1).fill(1))

  df2 = n2.times.map do
    [normal_rand(mu2[0], sigma), normal_rand(mu2[1], sigma)]
  end
  df2 = df2.transpose
  df2 = Daru::DataFrame.new(x: df2[0], y: df2[1], type: Array.new(n2).fill(0))

  df = df1.concat(df2)
  df = df.reindex(Daru::Index.new(df.index.to_a.shuffle))
  df[:index] = (n1 + n2).times.to_a
  df.set_index(:index)
end

# ロジスティック回帰
def run_logistic(train_set, plot)
  w = Numo::NArray[[0], [0.1], [0.1]]
  phi = train_set[:x, :y]
  phi[:bias] = Array.new(phi.size).fill(1)
  t = train_set[:type]
  t = t.to_matrix

  # 最大100回のIterationを実施
  100.times do
    # IRLS法によるパラメータの修正
    y = []
    phi.each_row do |line|
      a = Vector[*line.to_a].dot(w)
      y << (1.0 / (1.0 + Math.exp(-a)))
    end
    r = Matrix.diagonal(*(Numo::NArray[*y] * (1 - Numo::NArray[*y]).to_a))
    y = Numo::NArray[y].transpose
    tmp1 = Matrix[*Numo::NArray[*phi.to_matrix.transpose.to_a].dot(Numo::NArray[*r.to_a]).dot(Numo::NArray[*phi.to_matrix.to_a]).to_a].inverse
    tmp2 = Numo::NArray[*phi.to_matrix.transpose.to_a].dot(y - Numo::NArray[*t.transpose.to_a])
    w_new = w - Numo::NArray[*tmp1.to_a].dot(tmp2)

    # パラメータの変化が 0.1% 未満になったら終了
    if (w_new - w).transpose.dot(w_new - w).flatten[0] < (0.001 * (w.transpose.dot(w))).flatten[0]
      w = w_new
      break
    end
  end

  # 分類誤差の計算と確率付きデータの用意
  d0 = w[0]
  dx = w[1]
  dy = w[2]
  err = 0
  train_set[:probability] = Array.new(train_set.size).fill(0.0)

  train_set.each_row_with_index do |line, index|
    a = Vector[1, line.x, line.y].dot(w)
    p = 1.0 / (1.0 + Math.exp(-a))
    v = train_set.row[index]
    v[:probability] = p
    train_set.set_row_at([index], v)
    if (p - 0.5) * (line[:type] * 2 - 1) < 0
      err += 1
    end
  end

  err_rate = err * 100 / train_set.size

  # 境界線（P=0.5）を表示
  xmin = train_set.x.min - 5
  xmax = train_set.x.max + 5
  linex = Numo::NArray[*(xmin.to_i-5..xmax.to_i+4).to_a]
  liney = -linex * dx / dy - d0 / dy
  label = "ERR %.2f%%" % err_rate

  line_err = plot.add(:line, linex.cast_to(Numo::Int64).to_a, liney.cast_to(Numo::Int64).to_a)
  line_err.title(label)
  line_err.color('blue')

  # 確率付きデータを返却
  sorted_train_set = train_set.sort([:probability], ascending: false)
  sorted_train_set[:index] = sorted_train_set.size.times.to_a
  sorted_train_set.set_index(:index)
end

def run_simulation(variance, plot)
  train_set = prepare_dataset(variance)
  train_set1 = train_set.filter_rows {|row| row[:type] == 1 }
  train_set2 = train_set.filter_rows {|row| row[:type] == 0 }
  ymin = train_set.y.min - 5
  xmin = train_set.x.min - 5
  ymax = train_set.y.max + 10
  xmax = train_set.x.max + 10
  plot.configure do
    x_label("Variance: #{variance}")
    y_label('')
    xrange([xmin, xmax])
    yrange([ymin, ymax])
    legend(true)
    height(300)
    width(490)
  end

  scatter_true = plot.add_with_df(train_set1.to_nyaplotdf, :scatter, :x, :y)
  scatter_true.color('green')
  scatter_true.title('1')
  scatter_false = plot.add_with_df(train_set2.to_nyaplotdf, :scatter, :x, :y)
  scatter_false.color('orange')
  scatter_false.title('0')

  run_logistic(train_set, plot)
end

def draw_roc(result, plot)
  positives = result.filter_rows {|row| row[:type] == 1 }.size
  negatives = result.filter_rows {|row| row[:type] == 0 }.size
  tp = result.size.times.map { 0.0 }
  fp = result.size.times.map { 0.0 }
  result.each_row_with_index do |line, index|
    result.size.times do |c|
      if index < c
        if line[:type] == 1
          tp[c] += 1
        else
          fp[c] += 1
        end
      end
    end
  end
  
  tp_rate = Numo::NArray[*tp] / positives
  fp_rate = Numo::NArray[*fp] / negatives

  plot.configure do
    x_label('False positive rate')
    y_label('True positive rate')
    xrange([0, 1])
    yrange([0, 1])
    height(300)
    width(400)
  end

  plot.add(:line, fp_rate, tp_rate)
end

fig = Nyaplot::Frame.new

Variances.each do |variance|
  plot = Nyaplot::Plot.new
  result = run_simulation(variance, plot)
  fig.add(plot)

  plot = Nyaplot::Plot.new
  draw_roc(result, plot)
  fig.add(plot)
end

fig.show


<main>:6: warning: already initialized constant Variances
<main>:6: warning: previous definition of Variances was here


#<Nyaplot::Frame:0x007f97f7607728 @properties={:panes=>[#<Nyaplot::Plot:0x007f97f76075e8 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007f97f6ce17c0 @properties={:type=>:scatter, :options=>{:x=>:x, :y=>:y, :color=>"green", :title=>"1"}, :data=>"91c8e96d-b9e1-4459-98cc-c2341f35c465"}, @xrange=[-7.326129132067443, 31.192973909177297], @yrange=[-7.417671845661378, 26.08117761167197]>, #<Nyaplot::Diagram:0x007f97f75d5340 @properties={:type=>:scatter, :options=>{:x=>:x, :y=>:y, :color=>"orange", :title=>"0"}, :data=>"e8aedb91-dfcf-40fb-aebb-d8989598c227"}, @xrange=[-21.211040706393078, 17.310547348269147], @yrange=[-19.890492923642288, 14.757890304079758]>, #<Nyaplot::Diagram:0x007f97f74d1d68 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1", :title=>"ERR 18.00%", :color=>"blue"}, :data=>"67f63489-8cbb-45fd-981b-c6f75eeee8e3"}, @xrange=[-31, 40], @yrange=[-40, 31]>], :options=>{:x_label=>"Variance: 50", :y_label=>"", :xrange=>[-26.211040706393078, 41.1929739091773], :yrange=>[-24.890492923642288, 36.08117761167197], :legend=>true, :height=>300, :width=>490, :zoom=>true}}>, #<Nyaplot::Plot:0x007f97f6ba8c00 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007f97f74d2a88 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1"}, :data=>"8545b7fa-b52e-4d7b-826b-d675a78600df"}, @xrange=[0.0, 0.995], @yrange=[0.0, 1.0]>], :options=>{:x_label=>"False positive rate", :y_label=>"True positive rate", :xrange=>[0, 1], :yrange=>[0, 1], :height=>300, :width=>400, :zoom=>true}}>, #<Nyaplot::Plot:0x007f97f74d21a0 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007f97f76ef000 @properties={:type=>:scatter, :options=>{:x=>:x, :y=>:y, :color=>"green", :title=>"1"}, :data=>"41908590-b07e-4161-aeda-571dbc9e40f8"}, @xrange=[-19.052889239710158, 47.09088110856271], @yrange=[-27.2424254075547, 37.251231941646566]>, #<Nyaplot::Diagram:0x007f97f6f03918 @properties={:type=>:scatter, :options=>{:x=>:x, :y=>:y, :color=>"orange", :title=>"0"}, :data=>"3e8e2e54-48fd-492b-982c-7b0e5946d6ed"}, @xrange=[-34.24115251664867, 34.68629578427543], @yrange=[-33.44992445705856, 31.06635530945708]>, #<Nyaplot::Diagram:0x007f97f751cb10 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1", :title=>"ERR 33.00%", :color=>"blue"}, :data=>"5a8fb1e9-9d14-4414-a2e7-fe5959d32e6e"}, @xrange=[-44, 56], @yrange=[-56, 44]>], :options=>{:x_label=>"Variance: 150", :y_label=>"", :xrange=>[-39.24115251664867, 57.09088110856271], :yrange=>[-38.44992445705856, 47.251231941646566], :legend=>true, :height=>300, :width=>490, :zoom=>true}}>, #<Nyaplot::Plot:0x007f97f6b98968 @properties={:diagrams=>[#<Nyaplot::Diagram:0x007f97f7748d58 @properties={:type=>:line, :options=>{:x=>"data0", :y=>"data1"}, :data=>"694a7202-0709-452a-aa2b-08f7e9dc5197"}, @xrange=[0.0, 0.995], @yrange=[0.0, 1.0]>], :options=>{:x_label=>"False positive rate", :y_label=>"True positive rate", :xrange=>[0, 1], :yrange=>[0, 1], :height=>300, :width=>400, :zoom=>true}}>], :data=>{"91c8e96d-b9e1-4459-98cc-c2341f35c465"=>#<Nyaplot::DataFrame:0x007f97f6ce77b0 @name="91c8e96d-b9e1-4459-98cc-c2341f35c465", @rows=[{:type=>1, :x=>12.504269738762757, :y=>-2.3507547472963743}, {:type=>1, :x=>16.79859188309675, :y=>0.9905534274209842}, {:type=>1, :x=>14.060965626861943, :y=>7.176816798278052}, {:type=>1, :x=>8.872023951294816, :y=>2.7902459752307758}, {:type=>1, :x=>15.814209999195281, :y=>9.288124248041782}, {:type=>1, :x=>18.03281126452803, :y=>17.66245014694835}, {:type=>1, :x=>3.0647760394610613, :y=>25.526231150387474}, {:type=>1, :x=>13.690508198882997, :y=>-2.448959323984436}, {:type=>1, :x=>6.375877979992639, :y=>5.749756580132745}, {:type=>1, :x=>7.226076042592186, :y=>14.445529868501708}, {:type=>1, :x=>-1.1173609033513419, :y=>8.684024238975514}, {:type=>1, :x=>23.621531464038398, :y=>13.389416409237931}, {:type=>1, :x=>5.862040717011711, :y=>12.230439086670971}, {:type=>1, :x=>22.67831607665378, :y=>8.57718730651472}, {:type=>1, :x=>7.133577409927846, :y=>3.8157298453314974}, {:type=>1, :x=>2.761428544